In [21]:
import cifar_gpu_fun
reload(cifar_gpu_fun)
from cifar_gpu_fun import *
from numba import jit
import bcd
import sys
from sklearn.cluster import KMeans
from sklearn import metrics
import pyclust
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


#### Define Some Constants 

In [37]:
NUM_FEATURE_BATCHES=1
DATA_BATCH_SIZE=(1280)
FEATURE_BATCH_SIZE=(1024)
NUM_TRAIN = 50000
NUM_TEST = 10000
NUM_CLASSES = 10
NUM_FEATURES = FEATURE_BATCH_SIZE * NUM_FEATURE_BATCHES * 8
POOL_TYPE ='avg'
FILTER_GEN ='empirical'
BANDWIDTH = 1.0
LAMBDAS = [0.01, 0.1, 1.0, 0.001]
#LAMBDAS = [0.01]
CUDA_CONVNET = True
SCALE = 55.0
BIAS = 1.25
MIN_VAR_TOL = 1e-4
TOT_FEAT = FEATURE_BATCH_SIZE*NUM_FEATURE_BATCHES
NAN = float('nan')
saved = 0

In [3]:
np.random.seed(10)                                                                                                                                                                   
(XTrain, labelsTrain), (XTest, labelsTest) = load_cifar_processed()
patches = patchify_all_imgs(XTrain, (6,6), pad=False)

(50000, 3, 32, 32)


In [4]:
@jit(nopython=True)
def rbf(K, gamma):
    for x in range(K.shape[0]):
        for y in range(K.shape[1]):
            K[x,y] = math.exp(gamma*K[x,y])
            
    return K

@jit(nopython=True)
def arccos(K, XTrainNorms, XTestNorms):
    for x in range(K.shape[0]):
        for y in range(K.shape[1]):
            testNorm = np.sqrt(XTestNorms[x])
            trainNorm = np.sqrt(XTrainNorms[y]) 
            if (trainNorm == 0):
                trainNorm = 1
            if (testNorm == 0):
                testNorm = 1 
            cos = min(K[x,y]/(testNorm*trainNorm), 1)
            theta = np.arccos(cos)
            J = math.sin(theta) - (math.pi - theta)*math.cos(theta)
            K[x,y] = 1.0/math.pi * J
    return K
    


def computeRBFGramMatrix(K, XTrainNorms, XTestNorms, gamma=1, gamma_sample=10000):
    
    XTrainNorms = XTrainNorms.reshape(XTrainNorms.shape[0], 1)
    XTestNorms = XTestNorms.reshape(XTestNorms.shape[0], 1)
    print "TURNING K -> DISTANCE"
    K *= -2
    K += XTrainNorms.T
    K += XTestNorms 
    if (gamma == None):
        print "Calculating gamma"
        samples = numpy.random.choice(K.shape[0], gamma_sample*2, replace=False)
        x1 = samples[:gamma_sample]
        x2 = samples[gamma_sample:]
        sample_d = K[x1, x2]
        print "Sample d shape ", sample_d.shape
        median = numpy.median(sample_d)
        gamma = 2.0/median
        print gamma
    gamma = -1.0 * gamma

    print "Computing RBF"
    return rbf(K, gamma), -1.0*gamma

def saveGramMatrix(KTrain, KTest, save_id=saved):
    global saved
    KFile = open("/data/vaishaal/cifar_kernels/cifar_kernel_{0}".format(save_id), "w+")
    np.savez(KFile, train=KTrain, test=KTest)
    KFile.close()
    saved += 1
    
def computeDistanceMatrix(XTest, XTrain):
    XTrain = XTrain.reshape(XTrain.shape[0], -1)
    XTest = XTest.reshape(XTest.shape[0], -1)
    XTrain_norms = (np.linalg.norm(XTrain, axis=1) ** 2)[:, np.newaxis]
    XTest_norms = (np.linalg.norm(XTest, axis=1) ** 2)[:, np.newaxis]
    K = XTest.dot(XTrain.T)
    K *= -2
    K += XTrain_norms.T
    K += XTest_norms  
    return K
    
notebook_stdout = sys.stdout
notebook_stderr = sys.stderr
terminal_stdout = open('/dev/stdout', 'w')
terminal_stderr = open('/dev/stderr', 'w')

In [35]:
def make_empirical_filter_gen_clustering(patches, labels, sampling=8, MIN_VAR_TOL=0, k=128, seed=0, num_base_filters=16384*2):
    np.random.seed(seed)
    patches = patches.reshape(patches.shape[0]*patches.shape[1],*patches.shape[2:])
    all_idxs = np.random.choice(patches.shape[0], patches.shape[0], replace=False)
    curr_idx = [0]
    clustered_indices = [] 
    curr_clustered_idx = [0]
    def empirical_filter_gen(num_filters):
        clustered_left = (len(clustered_indices) - curr_clustered_idx[0])
        while (clustered_left - num_filters <= 0):
            uncluster_idxs = all_idxs[curr_idx[0]:curr_idx[0]+(num_filters*sampling)]
            curr_idx[0] += num_filters*sampling
            unclustered = patches[uncluster_idxs].astype('float32')
            clustered_indices.extend(kmeans_cluster(unclustered, uncluster_idxs, k=k))
            clustered_left = (len(clustered_indices) - curr_clustered_idx[0])
            print(clustered_left)

            
        idxs = clustered_indices[curr_clustered_idx[0]:curr_clustered_idx[0]+num_filters]
        curr_clustered_idx[0] += num_filters
        unfiltered = patches[idxs].astype('float32').transpose(0,3,1,2)
        old_shape = unfiltered.shape
        unfiltered = unfiltered.reshape(unfiltered.shape[0], -1)
        print(unfiltered.shape)
        unfiltered_vars = np.var(unfiltered, axis=1)
        filtered = unfiltered[np.where(unfiltered_vars > MIN_VAR_TOL)]
        out = np.ascontiguousarray(filtered[:num_filters].reshape(num_filters, *old_shape[1:]).copy())
        return out
    return empirical_filter_gen

def kmediods_cluster(data, indices, k=1024, seed=0):
    print "K={0}".format(k)
    print data.shape[0]
    shp = data.shape
    data = data.reshape(shp[0], -1)
    data = np.ascontiguousarray(data)
    ddt = data.dot(data.T)
    pos = np.maximum(ddt - 1.0, 0)
    neg = np.maximum(-1.0 * ddt - 1.0, 0)
    featurized_patches = np.hstack((pos, neg))
    print("RUNNING KMEDIODS")
    kmd = pyclust.KMedoids(n_clusters=k, n_trials=50)
    kmd.fit(featurized_patches)
    centers = kmd.centers_
    clustered_indices = []
    print centers.shape
    print featurized_patches.shape
    print("FINDING INDICES")
    for c in range(k):
        for d in range(data.shape[0]):
            if np.all(np.isclose(featurized_patches[d, :], centers[c, :])):
                clustered_indices.append(d)
                continue
    return np.array(clustered_indices)


def kmeans_cluster(data, indices, k=1024, seed=0):
    print "K={0}".format(k)
    print data.shape[0]
    shp = data.shape
    data = data.reshape(shp[0], -1)
    data = np.ascontiguousarray(data)
    ddt = data.dot(data.T)
    pos = np.maximum(ddt - 1.0, 0)
    neg = np.maximum(-1.0 * ddt - 1.0, 0)
    featurized_patches = np.hstack((pos, neg))
    print("RUNNING KMEANS")
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(featurized_patches)
    centers = kmeans.cluster_centers_
    clustered_indices = []
    print centers.shape
    print featurized_patches.shape
    print("FINDING INDICES")
    d = metrics.pairwise.pairwise_distances(centers, featurized_patches)
    clustered_indices = []
    for c in range(k):
        clustered_indices.append(np.argmin(d[c, :])) 
    return np.array(clustered_indices)
        
    
    
def greedy_cluster(filters, data, indices, k=10):
    shp = data.shape
    data = data.reshape(shp[0], -1)
    data = np.ascontiguousarray(data)
    ddt = data.dot(data.T)
    pos = np.maximum(ddt - 1.0, 0)
    neg = np.maximum(-1.0 * ddt - 1.0, 0)
    featurized_patches = np.hstack((pos, neg))
    distance_matrix = computeDistanceMatrix(featurized_patches, featurized_patches)
    distance_matrix[np.diag_indices(distance_matrix.shape[0])] = float('inf')
    
    skip = set()
    not_skip = []
    print("computed distance matrix")
    for row in range(distance_matrix.shape[0]):
        if row in skip:
            continue
        not_skip.append(row)
        sorted_indices = np.argsort(distance_matrix[row, :]) 
        top_k = sorted_indices[:k]
        distance_matrix[:, top_k] = float('inf')
        for r in top_k:
            skip.add(r)
            
    clustered_indices = indices[np.array(not_skip)]
    return clustered_indices

    

In [6]:
patches_flattened = patches.reshape(patches.shape[0]*patches.shape[1], *patches.shape[2:])

In [33]:
np.random.seed(0)
idxs = np.random.choice(patches_flattened.shape[0], 1024, replace=False)
sampled = patches_flattened[idxs, :]

In [34]:
clustered_idxs = kmeans_cluster(sampled, idxs, k=128)

K=128
1024
RUNNING KMEANS
(128, 2048)
(1024, 2048)
FINDING INDICES


In [29]:
clustered_idxs

array([  75,  739,   27,  286,   76,   95,  421,  277,  948,   67,  482,
        393,  872,  411,  928,  235,  455,  650,  850,  154,  566,  291,
        604,  898,  249,  113,  586,   19,   63,  195,  874,  790,  970,
        193,  963,  798,  275,   62,  575,  977,  904,  423,  125,   71,
        106,  513,  565, 1023,  992,  593,  437,  709,  848,    0,  676,
        134,  618,  192,  364,  774,  720,  680,  981,  177,  555,  521,
        768,  882,  828,  209,  415,  215,   33,  693,  986,  870,  909,
       1017,  973,  347,  294,  562,  122,  974,  381,  233,   17,  991,
        714,  897,  979,    8,  605,  227,  114,  626, 1001,  965,  976,
        408,  149,   37,  630,  180,  728,  922,  343,  617,  317,  414,
        572,  959, 1004,  544,  316,  846,  497,  988,  547,   30,  501,
        663,  730,  267,  530,  108,  244,  888])

In [138]:
filter_gen = make_empirical_filter_gen_clustering(patches, labelsTrain,  sampling=4, MIN_VAR_TOL=0, seed=3)

In [170]:
clustered_idxs.shape

(1024,)

In [15]:
VALID_3_POOLS = [(15,6)]

In [16]:
results = {}

In [38]:
sys.stdout = terminal_stdout
sys.stderr = terminal_stderr
#sys.stdout = notebook_stdout
#sys.stderr = notebook_stderr
accs = []
results = {}
for param in VALID_3_POOLS:
    pool_size, pool_stride = param
    filter_gen = make_empirical_filter_gen_clustering(patches, labelsTrain, sampling=1, k=128, MIN_VAR_TOL=0, seed=3)
    KTrain, KTest, normsTrain, normsTest, filters = convolveAndAccumulateGramAsync(XTrain, XTest, 
                                                                     labelsTrain, labelsTest, filter_gen,
                                                                     num_feature_batches=NUM_FEATURE_BATCHES, pool_size=pool_size, pool_stride=pool_stride, sync=40)
    
    result = {}
    for reg in LAMBDAS:
        acc = trainAndEvaluateDualModel(KTrain, KTest, labelsTrain, labelsTest, reg=reg)
        print "RESULT ", pool_size,pool_stride, reg, acc
        result[reg] = acc
    results[param] = result


KeyboardInterrupt: 

In [184]:
sys.stdout = terminal_stdout
sys.stderr = terminal_stderr
#sys.stdout = notebook_stdout
#sys.stderr = notebook_stderr
accs = []
results_control = {}
for param in VALID_3_POOLS:
    pool_size, pool_stride = param
    filter_gen = make_empirical_filter_gen(patches, labelsTrain, MIN_VAR_TOL=0, seed=3)
    KTrain, KTest, normsTrain, normsTest, filters = convolveAndAccumulateGramAsync(XTrain, XTest, 
                                                                     labelsTrain, labelsTest, filter_gen,
                                                                     num_feature_batches=NUM_FEATURE_BATCHES, pool_size=pool_size, pool_stride=pool_stride, sync=40)
    
    result = {}
    for reg in LAMBDAS:
        acc = trainAndEvaluateDualModel(KTrain, KTest, labelsTrain, labelsTest, reg=reg)
        print "RESULT ", pool_size,pool_stride, reg, acc
        result[reg] = acc
        result['cluster'] = False
    results_control[param] = result


In [20]:
sys.stdout = notebook_stdout
sys.stderr = notebook_stderr
print results

{}


In [188]:
print results_control

{(15, 6): {'cluster': False, 0.001: (1.0, 0.78339999999999999), 0.1: (0.95955999999999997, 0.81930000000000003), 0.01: (0.99950000000000006, 0.82479999999999998), 1.0: (0.83904000000000001, 0.76970000000000005)}}


In [178]:
gammas = [0.005]
regs = [0]
accs = []

In [16]:
sys.stdout = notebook_stdout
sys.stderr = notebook_stderr
for gamma in gammas:
    print "GAMMA"
    KTrainRBF = KTrain.copy()
    KTestRBF = KTest.copy()
    KTrainRBF, train_gamma = computeRBFGramMatrix(KTrainRBF, normsTrain, normsTrain, gamma=gamma)
    KTestRBF, _ = computeRBFGramMatrix(KTestRBF, normsTrain, normsTest, gamma=train_gamma)
    for reg in regs:
        try:
            print("Starting Train","reg", reg, "gamma", gamma)
            sys.stdout.flush()
            %time train,test = trainAndEvaluateDualModel(KTrainRBF, KTestRBF, labelsTrain, labelsTest, reg=reg)
            print train,test,1,reg,gamma
            sys.stdout.flush()
            accs.append((gamma,reg,train,test))
        except:
            print("Singular Matrix")
            accs.append(0)


GAMMA
TURNING K -> DISTANCE
Computing RBF
TURNING K -> DISTANCE
Computing RBF
('Starting Train', 'reg', 0, 'gamma', 0.005)
Learning Dual Model better
REG IS  0
CPU times: user 43min 51s, sys: 33 s, total: 44min 24s
Wall time: 2min 16s
1.0 0.8169 1 0 0.005


In [ ]:
sys.stdout = notebook_stdout
sys.stderr = notebook_stderr
for i, acc in enumerate(accs):
    gamma,reg,train, test = acc
    print train,test,1,reg,gamma

In [91]:
KTrain_128 = np.load("/data/vaishaal/KTrain_128.npy")
KTest_128 = np.load("/data/vaishaal/KTest_128.npy")
normsTrain_128 = np.load("/data/vaishaal/norms_train_128.npy")
normsTest_128 = np.load("/data/vaishaal/norms_test_128.npy")

In [92]:
KTrain_fuse = KTrain_128 + KTrain
KTest_fuse = KTest_128 + KTest
normsTrain_fuse = normsTrain_128 + normsTrain
normsTest_fuse = normsTest_128 + normsTest

In [102]:
acc1 = trainAndEvaluateDualModel(KTrain, KTest, labelsTrain, labelsTest, reg=0.01)

Learning Dual Model better
REG IS  0.01


In [103]:
acc1

(0.99973999999999996, 0.83530000000000004)

In [99]:
acc2 = trainAndEvaluateDualModel(KTrain_fuse, KTest_fuse, labelsTrain, labelsTest, reg=0.05)
print acc2

Learning Dual Model better
REG IS  0.05
(0.99883999999999995, 0.82879999999999998)


In [96]:
print acc2

(1.0, 0.83109999999999995)


In [40]:
np.save("/data/vaishaal/KTrain_1024_patches", KTrain)

In [39]:
np.save("/data/vaishaal/KTest_1024_patches", KTest)

In [38]:
np.save("/data/vaishaal/norms_train_1024_patches", normsTrain)

In [37]:
np.save("/data/vaishaal/norms_test_1024_patches", normsTest)

In [23]:
np.save("/data/vaishaal/labels_train", labelsTrain)

In [24]:
np.save("/data/vaishaal/labels_test", labelsTest)

In [103]:
del KTrain_128

In [104]:
del KTrain_fuse

In [25]:
KTrain.shape

(50000, 50000)

In [16]:
patches

(50000, 729, 6, 6, 3)

In [72]:
KTrainRBF = KTrain.copy()
KTestRBF = KTest.copy()
def kernel_to_distance(K, XTrainNorms, XTestNorms):
    XTrainNorms = XTrainNorms.reshape(XTrainNorms.shape[0], 1)
    XTestNorms = XTestNorms.reshape(XTestNorms.shape[0], 1)
    print "TURNING K -> DISTANCE"
    K *= -2
    K += XTrainNorms.T
    K += XTestNorms 
    return K

def estimate_gamma(D,gamma_sample=10000):
    print "Calculating gamma"
    samples = numpy.random.choice(D.shape[0], gamma_sample*2, replace=False)
    x1 = samples[:gamma_sample]
    x2 = samples[gamma_sample:]
    sample_d = D[x1, x2]
    print "Sample d shape ", sample_d.shape
    median = np.median(sample_d)
    gamma = 2.0/median
    return gamma
D = kernel_to_distance(KTrainRBF, normsTrain, normsTrain)

TURNING K -> DISTANCE


In [91]:
pts = np.random.choice(D.shape[0], 1000*2, replace=False)
x1 = pts[:1000]
x2 = pts[1000:]
sample_d = D[x1, x2]
print "Sample d shape ", sample_d.shape
median = np.median(sample_d)
gamma = 2.0/median


Sample d shape  (1000,)


In [101]:
KTrain = KTrainRBF

array([  0.00000000e+00,   7.62939453e-06,   0.00000000e+00, ...,
         1.90734863e-06,   0.00000000e+00,   0.00000000e+00])

In [95]:
KTrainRBF

array([[ 33.20146561,  36.28236389,  15.55484772, ...,  17.90564346,
         25.23268318,  29.74692154],
       [ 36.28236389,  44.59234619,  17.80164909, ...,  20.59594727,
         30.45558167,  34.82453918],
       [ 15.55484772,  17.80164909,   9.16023064, ...,   9.84381771,
         11.05340576,  15.00150013],
       ..., 
       [ 17.90564346,  20.59594727,   9.84381771, ...,  11.95548248,
         13.03578663,  17.35746956],
       [ 25.23268318,  30.45558167,  11.05340576, ...,  13.03578663,
         25.01102257,  24.62996674],
       [ 29.74692154,  34.82453918,  15.00150013, ...,  17.35746956,
         24.62996674,  31.18713188]])

In [97]:
KTrain

array([[  0.00000000e+00,   5.22908783e+00,   1.12520008e+01, ...,
          9.34566212e+00,   7.74712181e+00,   4.89475441e+00],
       [  5.22908783e+00,   7.62939453e-06,   1.81492825e+01, ...,
          1.53559389e+01,   8.69220924e+00,   6.13040352e+00],
       [  1.12520008e+01,   1.81492825e+01,   0.00000000e+00, ...,
          1.42807865e+00,   1.20644417e+01,   1.03443623e+01],
       ..., 
       [  9.34566212e+00,   1.53559389e+01,   1.42807865e+00, ...,
          1.90734863e-06,   1.08949327e+01,   8.42767620e+00],
       [  7.74712181e+00,   8.69220924e+00,   1.20644417e+01, ...,
          1.08949327e+01,   0.00000000e+00,   6.93822098e+00],
       [  4.89475441e+00,   6.13040352e+00,   1.03443623e+01, ...,
          8.42767620e+00,   6.93822098e+00,   0.00000000e+00]])

In [104]:
KTrain

array([[ 33.20146561,  36.28236389,  15.55484772, ...,  17.90564346,
         25.23268318,  29.74692154],
       [ 36.28236389,  44.59234619,  17.80164909, ...,  20.59594727,
         30.45558167,  34.82453918],
       [ 15.55484772,  17.80164909,   9.16023064, ...,   9.84381771,
         11.05340576,  15.00150013],
       ..., 
       [ 17.90564346,  20.59594727,   9.84381771, ...,  11.95548248,
         13.03578663,  17.35746956],
       [ 25.23268318,  30.45558167,  11.05340576, ...,  13.03578663,
         25.01102257,  24.62996674],
       [ 29.74692154,  34.82453918,  15.00150013, ...,  17.35746956,
         24.62996674,  31.18713188]])

In [105]:
normsTrain

array([ 33.20146561,  44.59235001,   9.16023064, ...,  11.95548344,
        25.01102257,  31.18713188], dtype=float32)